### Fine-tuning LLM to detect Fallacies in Arguments

Using data from:

Ruiz-Dolz, R., & Lawrence, J. (2023, December). Detecting Argumentative Fallacies in the Wild: Problems and Limitations of Large Language Models. In M. Alshomary, C.-C. Chen, S. Muresan, J. Park, & J. Romberg (Eds.), Proceedings of the 10th Workshop on Argument Mining (pp. 1–10). Retrieved from https://aclanthology.org/2023.argmining-1.1

https://github.com/raruidol/ArgumentMining23-Fallacy/blob/main/README.md




In [4]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_fscore_support
import evaluate
import torch
import numpy as np
import json


In [3]:
try:
      with open("fallacy_corpus.json") as filehandle:
          json_data = json.load(filehandle)
except:
      print('The file is not available.')
      exit()

In [5]:
data = {'train': {}, 'dev': {}, 'test': {}}

data['train']['label'] = []
data['train']['text'] = []
data['dev']['label'] = []
data['dev']['text'] = []
data['test']['label'] = []
data['test']['text'] = []

fallacyStringConversions = {
'None': 0,
'AdHominem': 1,
'AppealtoEmotion': 2, 
'AppealtoAuthority': 3, 
'Slipperyslope': 4,
'AppealtoMajority': 5
}

fallacyNumberConversions = {
0: 'None',
1: 'AdHominem',
2: 'AppealtoEmotion', 
3: 'AppealtoAuthority', 
4: 'Slipperyslope',
5: 'AppealtoMajority'
}

for sample in json_data['train']:
    data['train']['text'].append(sample[0])
    data['train']['label'].append(fallacyStringConversions[sample[1]])

for sample in json_data['dev']:
        data['dev']['text'].append(sample[0])
        data['dev']['label'].append(fallacyStringConversions[sample[1]])

for sample in json_data['test']:
        data['test']['text'].append(sample[0])
        data['test']['label'].append(fallacyStringConversions[sample[1]])


final_data = DatasetDict()
for k, v in data.items():
    final_data[k] = Dataset.from_dict(v)
    

Create Model

In [6]:
model_checkpoint = 'distilbert-base-uncased'
# model_checkpoint = 'roberta-base' # you can alternatively use roberta-base but this model is bigger thus training will take longer


# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained( model_checkpoint, num_labels=6, ignore_mismatched_sizes=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Pre-process data 

In [7]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [8]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [9]:
# tokenize training and validation datasets
tokenized_dataset = final_data.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████| 270/270 [00:00<00:00, 11446.99 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 2152
    })
    dev: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 266
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 270
    })
})

In [10]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Evaluate Model

In [11]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [12]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

Train Model

In [13]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [14]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 632,070 || all params: 67,590,156 || trainable%: 0.9352


In [15]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [16]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

In [19]:
# train model
trainer.train()

  9%|▉         | 501/5380 [03:06<54:03,  1.50it/s]  

{'loss': 0.9636, 'grad_norm': 7.326727390289307, 'learning_rate': 0.0009070631970260224, 'epoch': 0.93}


 10%|█         | 538/5380 [03:19<21:35,  3.74it/s]  Trainer is attempting to log a value of "{'accuracy': 0.6992481203007519}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
                                                  
 10%|█         | 538/5380 [03:35<21:35,  3.74it/s]

{'eval_loss': 0.8581797480583191, 'eval_accuracy': {'accuracy': 0.6992481203007519}, 'eval_runtime': 16.2397, 'eval_samples_per_second': 16.38, 'eval_steps_per_second': 4.126, 'epoch': 1.0}


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
 19%|█▊        | 1001/5380 [05:21<10:01,  7.28it/s] 

...

100%|██████████| 5380/5380 [48:22<00:00,  4.68it/s]Trainer is attempting to log a value of "{'accuracy': 0.7330827067669173}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
                                                   
100%|██████████| 5380/5380 [48:36<00:00,  4.68it/s]

{'eval_loss': 2.0656931400299072, 'eval_accuracy': {'accuracy': 0.7330827067669173}, 'eval_runtime': 13.3405, 'eval_samples_per_second': 19.939, 'eval_steps_per_second': 5.022, 'epoch': 10.0}


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 5380/5380 [48:37<00:00,  1.84it/s]

{'train_runtime': 2917.0913, 'train_samples_per_second': 7.377, 'train_steps_per_second': 1.844, 'train_loss': 0.38968762146052816, 'epoch': 10.0}


TrainOutput(global_step=5380, training_loss=0.38968762146052816, metrics={'train_runtime': 2917.0913, 'train_samples_per_second': 7.377, 'train_steps_per_second': 1.844, 'total_flos': 599914265851008.0, 'train_loss': 0.38968762146052816, 'epoch': 10.0})

Make Predictions With Model

In [21]:

dev_predictions = trainer.predict(tokenized_dataset['dev'])
dev_predict = np.argmax(dev_predictions.predictions, axis=-1)
test_predictions = trainer.predict(tokenized_dataset['test'])
test_predict = np.argmax(test_predictions.predictions, axis=-1)



# mf1_dev = precision_recall_fscore_support(tokenized_dataset['dev']['label'], dev_predict, average='macro')
# mf1_test = precision_recall_fscore_support(tokenized_dataset['test']['label'], test_predict, average='macro')

# print('Score in, DEV:', mf1_dev, 'TEST:', mf1_test)
# print('Confusion matrix:')
# print(confusion_matrix(tokenized_dataset['test']['label'], test_predict))

100%|██████████| 68/68 [00:06<00:00, 11.07it/s]

[4 0 0 4 0 0 0 0 0 0 0 0 0 4 4 0 4 4 0 0 0 0 0 0 4 0 0 1 3 3 0 3 0 0 3 3 0
 0 0 0 3 3 0 0 3 3 0 0 0 3 3 0 3 3 1 3 3 0 0 0 0 0 3 0 4 3 3 0 3 1 5 0 3 0
 1 0 3 0 0 0 1 3 0 0 0 0 0 0 5 0 0 0 0 4 0 5 0 0 5 0 5 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 5 3 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 5 0 0 0 0 0 0
 0 0 5 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 5 0 0
 0 0 0 0 0 0 0]
[0 0 0 0 0 0 3 0 4 0 0 0 4 4 0 4 0 3 0 4 0 0 4 0 4 0 4 3 3 3 3 0 3 4 0 0 0
 3 0 0 0 0 3 0 3 3 0 1 0 0 3 3 0 0 0 3 0 3 3 3 0 0 3 5 3 0 0 0 0 0 3 0 0 1
 1 3 0 1 3 0 0 3 3 0 3 0 0 0 0 5 5 5 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 4
 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 5 0 5 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 3 0 0 0 0 0 0 0 0 0 0 0
 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 4 0 0 0 0 0 0
 0 0 0 0 

In [31]:
print("Actual\t| Predicted")
count = 0
for i, prediction in enumerate(dev_predict):
    if prediction == data['dev']['label'][i]:
        count += 1
    print(fallacyNumberConversions[data['dev']['label'][i]] + " | "+ fallacyNumberConversions[prediction])
print("Accuracy: " + str(count/len(dev_predict) * 100))

Actual	| Predicted
Slipperyslope | Slipperyslope
Slipperyslope | None
Slipperyslope | None
Slipperyslope | Slipperyslope
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | Slipperyslope
Slipperyslope | Slipperyslope
Slipperyslope | None
Slipperyslope | Slipperyslope
Slipperyslope | Slipperyslope
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | None
Slipperyslope | Slipperyslope
Slipperyslope | None
Slipperyslope | None
AppealtoAuthority | AdHominem
AppealtoAuthority | AppealtoAuthority
AppealtoAuthority | AppealtoAuthority
AppealtoAuthority | None
AppealtoAuthority | AppealtoAuthority
AppealtoAuthority | None
AppealtoAuthority | None
AppealtoAuthority | AppealtoAuthority
AppealtoAuthority | AppealtoAuthority
AppealtoAuthority | None
AppealtoAuthority | None
Appeal

Save Model

In [ ]:
from huggingface_hub import notebook_login
notebook_login() # ensure token gives write access

In [45]:
hf_name = 'lennon2020' 
model_id = hf_name + "/" + "lora-fallacy-classification" # you can name the model whatever you want

In [46]:
model.push_to_hub(model_id) # save model
trainer.push_to_hub(model_id) # save trainer

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
adapter_model.safetensors: 100%|██████████| 2.53M/2.53M [00:03<00:00, 821kB/s] 



adapter_model.safetensors:   0%|          | 0.00/2.53M [00:00<?, ?B/s]

training_args.bin: 100%|██████████| 5.18k/5.18k [00:00<00:00, 20.5kB/s]30MB/s]
events.out.tfevents.1716251357.Lennons-MacBook-Air.local.49143.0: 100%|██████████| 10.2k/10.2k [00:00<00:00, 50.1kB/s]
adapter_model.safetensors: 100%|██████████| 2.53M/2.53M [00:03<00:00, 759kB/s] 

Upload 3 LFS files: 100%|██████████| 3/3 [00:03<00:00,  1.26s/it]


CommitInfo(commit_url='https://huggingface.co/lennon2020/distilbert-base-uncased-lora-text-classification/commit/98f9ade3b7f9e4024d2ebd9cf70a5eea63d5bbc7', commit_message='lennon2020/lora-fallacy-classification', commit_description='', oid='98f9ade3b7f9e4024d2ebd9cf70a5eea63d5bbc7', pr_url=None, pr_revision=None, pr_num=None)